## POS tagging using modified Viterbi

### Data Preparation

In [354]:
#Importing libraries
import nltk
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [355]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\monis\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [356]:
nltk.corpus.universal_treebanks

<ConllCorpusReader in '.../corpora/universal_treebanks_v20' (not loaded yet)>

In [357]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [358]:
print(nltk_data[:40])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [359]:
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:40])

3718
196
[[('``', '.'), ('When', 'ADV'), ('you', 'PRON'), ('do', 'VERB'), ('that', 'DET'), ('*T*-1', 'X'), (',', '.'), ('there', 'DET'), ('is', 'VERB'), ('not', 'ADV'), ('a', 'DET'), ('cut', 'NOUN'), (',', '.'), ('but', 'CONJ'), ('there', 'DET'), ('is', 'VERB'), ('in', 'ADP'), ('fact', 'NOUN'), ('a', 'DET'), ('program', 'NOUN'), ('increase', 'NOUN'), ('of', 'ADP'), ('$', '.'), ('5', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ("''", '.'), ('each', 'DET'), ('for', 'ADP'), ('the', 'DET'), ('FTC', 'NOUN'), ('and', 'CONJ'), ('the', 'DET'), ('Justice', 'NOUN'), ('Department', 'NOUN'), (',', '.'), ('Rep.', 'NOUN'), ('Neal', 'NOUN'), ('Smith', 'NOUN'), ('-LRB-', '.'), ('D.', 'NOUN'), (',', '.'), ('Iowa', 'NOUN'), ('-RRB-', '.'), ('said', 'VERB'), ('0', 'X'), ('*T*-2', 'X'), ('during', 'ADP'), ('House', 'NOUN'), ('debate', 'NOUN'), ('.', '.')], [('Many', 'ADJ'), ('grain', 'NOUN'), ('processors', 'NOUN'), ('and', 'CONJ'), ('exporters', 'NOUN'), ('use', 'VERB'), ('the', 'DET'), ('price', 'NOUN'), 

In [360]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95730

In [361]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['``', 'When', 'you', 'do', 'that', '*T*-1', ',', 'there', 'is', 'not']

In [362]:
# vocabulary
V = set(tokens)
print(len(V))

12095


In [363]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [365]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

#### Transition probablities

In [366]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [367]:
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

#### Lexical and Rule based taggers

In [418]:
unigram_tagger = nltk.UnigramTagger(train_set)
unigram_tagger.evaluate(test_set)

0.9049737161342499

In [368]:
def unigram_based_tagger(train_set, test_set):
    # Lexicon (or unigram tagger)
    unigram_tagger = nltk.UnigramTagger(train_set)
    unigram_tagger.evaluate(test_set)
    return unigram_tagger

In [416]:
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*ould$', 'VERB'),              # modals
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
    ]
regexp_tagger = nltk.RegexpTagger(patterns)
regexp_tagger.evaluate(test_set)


0.32713303679741207

In [414]:
def rule_based_tagger(test_set):
    patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*ould$', 'VERB'),              # modals
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
    ]
    regexp_tagger = nltk.RegexpTagger(patterns)
    regexp_tagger.evaluate(test_set)
    return regexp_tagger

#### Tags matrix creation to make dataframe out of it

In [370]:
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [371]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [372]:
test = rule_based_tagger(test_set)

### Build the vanilla Viterbi based POS tagger

In [373]:
rule_based_tagged = rule_based_tagger(test_set)

In [374]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    rule_tagged =[]
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
#         print(pmax)
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))


#### Modified Viterbi  by adding rule based tagger

In [375]:
# Viterbi Heuristic
def Viterbi_modified(words, train_bag = train_tagged_words):
    state = []
    rule_tagged = []
    T = list(set([pair[1] for pair in train_bag]))
    
     
    for key, word in enumerate(words):
        
        ## Applying rules for the unknown words which are not part of the Vo
        if word not in V:
            rule_tagged.append(rule_based_tagged.tag_sents([[word]])[0][0][1])
            state.append(rule_based_tagged.tag_sents([[word]])[0][0][1])
        
        else:
            p = []
            for tag in T:
#                 print(key)
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)
            
            pmax = max(p)
            state_max = T[p.index(pmax)] 
            state.append(state_max)
            
    
#     state.extend(rule_tagged)        
    return list(zip(words, state))


In [378]:
random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('Prosecutors', 'NOUN'),
  ('need', 'VERB'),
  ('court', 'NOUN'),
  ('permission', 'NOUN'),
  ('*-1', 'X'),
  ('to', 'PRT'),
  ('obtain', 'VERB'),
  ('the', 'DET'),
  ('tax', 'NOUN'),
  ('returns', 'NOUN'),
  ('of', 'ADP'),
  ('an', 'DET'),
  ('individual', 'NOUN'),
  ('or', 'CONJ'),
  ('a', 'DET'),
  ('business', 'NOUN'),
  ('.', '.')],
 [('In', 'ADP'),
  ('a', 'DET'),
  ('meeting', 'NOUN'),
  ('Tuesday', 'NOUN'),
  (',', '.'),
  ('supreme', 'NOUN'),
  ('leader', 'NOUN'),
  (',', '.'),
  ('Deng', 'NOUN'),
  ('Xiaoping', 'NOUN'),
  (',', '.'),
  ('told', 'VERB'),
  ('Mr.', 'NOUN'),
  ('Nixon', 'NOUN'),
  (',', '.'),
  ('``', '.'),
  ('*', 'X'),
  ('Frankly', 'ADV'),
  ('speaking', 'VERB'),
  (',', '.'),
  ('the', 'DET'),
  ('U.S.', 'NOUN'),
  ('was', 'VERB'),
  ('involved', 'VERB'),
  ('*-146', 'X'),
  ('too', 'ADV'),
  ('deeply', 'ADV'),
  ('in', 'ADP'),
  ('the', 'DET'),
  ('turmoil', 'NOUN'),
  ('and', 'CONJ'),
  ('counterrevolutionary', 'ADJ'),
  ('rebellion', 'NOUN'),
  ('which'

In [384]:
tagged_seq = Viterbi(test_tagged_words)

In [385]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.8548387096774194

In [386]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [387]:
incorrect_tagged_cases

[[('an', 'DET'), (('individual', 'ADJ'), ('individual', 'NOUN'))],
 [(',', '.'), (('supreme', 'ADJ'), ('supreme', 'NOUN'))],
 [(',', '.'), (('Deng', 'ADJ'), ('Deng', 'NOUN'))],
 [('Deng', 'NOUN'), (('Xiaoping', 'ADJ'), ('Xiaoping', 'NOUN'))],
 [('*', 'X'), (('Frankly', 'ADJ'), ('Frankly', 'ADV'))],
 [('Frankly', 'ADV'), (('speaking', 'ADJ'), ('speaking', 'VERB'))],
 [('involved', 'VERB'), (('*-146', 'ADJ'), ('*-146', 'X'))],
 [('too', 'ADV'), (('deeply', 'ADJ'), ('deeply', 'ADV'))],
 [('counterrevolutionary', 'ADJ'),
  (('rebellion', 'ADJ'), ('rebellion', 'NOUN'))],
 [('which', 'DET'), (('*T*-243', 'ADJ'), ('*T*-243', 'X'))],
 [('not', 'ADV'), (('long', 'ADJ'), ('long', 'ADV'))],
 [('long', 'ADV'), (('ago', 'ADP'), ('ago', 'ADV'))],
 [('are', 'VERB'), (('reaping', 'ADJ'), ('reaping', 'VERB'))],
 [('sudden', 'ADJ'), (('windfall', 'ADJ'), ('windfall', 'NOUN'))],
 [('far', 'ADV'), (('less', 'ADJ'), ('less', 'ADV'))],
 [('Japanese', 'ADJ'), (('penetration', 'ADJ'), ('penetration', 'NOUN'))

### Solve the problem of unknown words

In [388]:
unknown_words = set(test_tagged_words) - V

In [389]:
unknown_words

{'*-146',
 '*T*-243',
 'Deng',
 'Frankly',
 'Xiaoping',
 'counterrevolutionary',
 'deeply',
 'penetration',
 'rash',
 'reaping',
 'rebellion',
 'speaking',
 'supreme',
 'windfall'}

In [392]:
Viterbi(list(unknown_words))

[('speaking', 'ADJ'),
 ('*-146', 'ADJ'),
 ('Deng', 'ADJ'),
 ('rash', 'ADJ'),
 ('rebellion', 'ADJ'),
 ('deeply', 'ADJ'),
 ('supreme', 'ADJ'),
 ('*T*-243', 'ADJ'),
 ('penetration', 'ADJ'),
 ('Xiaoping', 'ADJ'),
 ('windfall', 'ADJ'),
 ('reaping', 'ADJ'),
 ('counterrevolutionary', 'ADJ'),
 ('Frankly', 'ADJ')]

In [390]:
Viterbi_modified(unknown_words)

[('speaking', 'VERB'),
 ('*-146', 'NOUN'),
 ('Deng', 'NOUN'),
 ('rash', 'NOUN'),
 ('rebellion', 'NOUN'),
 ('deeply', 'NOUN'),
 ('supreme', 'NOUN'),
 ('*T*-243', 'NOUN'),
 ('penetration', 'NOUN'),
 ('Xiaoping', 'VERB'),
 ('windfall', 'NOUN'),
 ('reaping', 'VERB'),
 ('counterrevolutionary', 'NOUN'),
 ('Frankly', 'NOUN')]

In [106]:
def parse_text(document):
    "parse text into words"
    return re.findall(r'\w+', document.lower())

In [248]:
test_sent = parse_text(open("Test_sentences.txt").read())

In [ ]:
Viterbi(test_sent)

In [274]:
Viterbi_modified(test_sent)

IndexError: list index out of range

#### Evaluating tagging accuracy

In [394]:
tagged_seq = Viterbi_modified(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.9112903225806451

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [398]:
tagged_seq = Viterbi(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.8548387096774194

In [399]:
tagged_seq = Viterbi_modified(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.9112903225806451

In [403]:
print(f"The difference between Viterbi modified and Viterbi Vanilla {round(0.91 - 0.85,2)*100}%")

The difference between Viterbi modified and Viterbi Vanilla 6.0%


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [397]:
vanila_viterbi_results = Viterbi(test_sent)

In [396]:
modified_viterbi_results = Viterbi_modified(test_sent)

#### Some of the Incorrected Tags

In [409]:
print(vanila_viterbi_results[0], vanila_viterbi_results[3], vanila_viterbi_results[4], vanila_viterbi_results[6])


('android', 'ADJ') ('mobile', 'ADJ') ('operating', 'NOUN') ('developed', 'VERB')


#### Some of the Corrected Tags after modifying Viterbi

In [412]:
print(modified_viterbi_results[0], modified_viterbi_results[3], modified_viterbi_results[4], vanila_viterbi_results[6])

('android', 'NOUN') ('mobile', 'ADJ') ('operating', 'NOUN') ('developed', 'VERB')
